# crawling_naver news

In [1]:
pwd

'G:\\내 드라이브\\Colab Notebooks\\00_python\\01_crawling'

### Step 0. 준비

In [2]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver import ActionChains as AC  # 웹 브라우저 동작
from tqdm import tqdm
from tqdm.notebook import tqdm
import re
from time import sleep
import time

### Step 1. 크롤링할 뉴스들 url 수집

In [3]:
# 검색어
keyword = input("크롤링할 키워드를 입력하세요: ")

크롤링할 키워드를 입력하세요: 테슬라


In [4]:
# 크롬 웹브라우저 실행
driver = webdriver.Chrome(r"./chromedriver/chromedriver.exe")
driver.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}".format(keyword))
time.sleep(2)

In [5]:
# page1에서 네이버뉴스 url 수집하기
things = driver.find_elements_by_link_text('네이버뉴스')  

url_list = []
for thing in things:
    url = thing.get_attribute('href')
    url_list.append(url)

print(len(url_list))
url_list

26


['https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=103&oid=030&aid=0002961270',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=003&aid=0010645653',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=003&aid=0010645419',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=001&aid=0012574350',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=025&aid=0003123935',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=008&aid=0004625860',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=009&aid=0004833181',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=421&aid=0005520374',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=018&aid=0005002083',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=014&aid=0004685815',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=104&oid=001&aid=0

In [6]:
# url_list 저장
df = pd.DataFrame({"url":url_list})
df.to_excel('naver_news_urls.xlsx')

### Step 2. 전체 기사 for문으로 데이터 수집

In [7]:
# 저장해둔 url 불러오기
df = pd.read_excel('naver_news_urls.xlsx')
print(len(df['url']))
df['url']

26


0     https://news.naver.com/main/read.naver?mode=LS...
1     https://news.naver.com/main/read.naver?mode=LS...
2     https://news.naver.com/main/read.naver?mode=LS...
3     https://news.naver.com/main/read.naver?mode=LS...
4     https://news.naver.com/main/read.naver?mode=LS...
5     https://news.naver.com/main/read.naver?mode=LS...
6     https://news.naver.com/main/read.naver?mode=LS...
7     https://news.naver.com/main/read.naver?mode=LS...
8     https://news.naver.com/main/read.naver?mode=LS...
9     https://news.naver.com/main/read.naver?mode=LS...
10    https://news.naver.com/main/read.naver?mode=LS...
11    https://news.naver.com/main/read.naver?mode=LS...
12    https://news.naver.com/main/read.naver?mode=LS...
13    https://news.naver.com/main/read.naver?mode=LS...
14    https://news.naver.com/main/read.naver?mode=LS...
15    https://news.naver.com/main/read.naver?mode=LS...
16    https://news.naver.com/main/read.naver?mode=LS...
17    https://news.naver.com/main/read.naver?mod

In [8]:
number = len(df['url'])
number

26

In [9]:
dict = {}

# 페이지당 기사 수집
for i in tqdm(range(0, number)):   # len(df['url'])
    try:
        # 뉴스 크롬창 띄우기
        driver = webdriver.Chrome(r"./chromedriver/chromedriver.exe")
        driver.get(df['url'][i])
        time.sleep(1)

        # 기사 데이터 수집
        title = driver.find_element_by_css_selector('.tts_head').text
        date = driver.find_element_by_css_selector('.t11').text
        up = driver.find_element_by_css_selector('.u_cnt._count').text
        like = driver.find_element_by_css_selector(".end_btn .u_likeit_list.good .u_likeit_list_count._count").text
        warm = driver.find_element_by_css_selector(".end_btn .u_likeit_list.warm .u_likeit_list_count._count").text
        sad = driver.find_element_by_css_selector(".end_btn .u_likeit_list.sad .u_likeit_list_count._count").text
        angry = driver.find_element_by_css_selector(".end_btn .u_likeit_list.angry .u_likeit_list_count._count").text
        want = driver.find_element_by_css_selector(".end_btn .u_likeit_list.want .u_likeit_list_count._count").text

        # 기사 댓글 갯수 
        review_count = int(driver.find_element_by_css_selector(".u_cbox_count").text.replace(',', ''))

        # 더보기 클릭 횟수
        moreview_num = review_count//20

        # 댓글 버튼 클릭
        driver.find_element_by_css_selector(".lo_txt").click()
        time.sleep(1)

        # 더보기 버튼 여러번 클릭하기
        k=0
        while k <= moreview_num:  # 더보기 횟수만큼 반복 
            try:
                driver.find_element_by_css_selector(".u_cbox_page_more").click()  # 더보기 버튼 클릭
                time.sleep(1)
                k = k+1
            except:        
                break   # 에러나면 클릭 반복문을 빠져나가라

        # review 수집하기
        review_list = []
        overlays1 = ".u_cbox_text_wrap"
        reviews = driver.find_elements_by_css_selector(overlays1)
        for review in tqdm(reviews):    
            review = review.text
            review_list.append(review)

        target_info = {}
        target_info['기사명'] = title
        target_info['날짜'] = date
        target_info['추천수'] = up
        target_info['좋아요'] = like
        target_info['훈훈해요'] = warm
        target_info['슬퍼요'] = sad
        target_info['화나요'] = angry
        target_info['후속기사 원해요'] = want
        target_info['댓글 수'] = len(review_list)
        target_info['댓글'] = review_list

        dict[i] = target_info
        
        print(title, '( 댓글 수:', len(review_list),')')

        driver.close()
        time.sleep(1)
        
    except:
        driver.close()
        continue

print(len(dict))
dict


벤츠 E클래스, 7월 '수입차 1위' 탈환…테슬라는 물량 급감 ( 댓글 수: 36 )



테슬라 7월 판매 22대로 급감…'벤츠 E클래스' 2567대로 1위(종합) ( 댓글 수: 21 )



지난달 수입차 판매 1위는 '벤츠 E클래스'…테슬라는 부진 왜 ( 댓글 수: 1 )



테슬라 주춤한 사이 벤츠 날았다…7월에도 수입차 질주 여전 ( 댓글 수: 1 )



테슬라 문 안 열려 '중대결함'…머스크 서울서 고발당했다 ( 댓글 수: 16 )


...


팀 쿡 “테슬라 팔아라” 머스크 “애플 CEO 넘기면” ( 댓글 수: 10 )



테슬라 설립 뒤 확보한 모든 특허 뜯어봤더니… [정영효의 일본산업 분석] ( 댓글 수: 7 )



3년간 60건…'AI 특허' 공들인 테슬라 ( 댓글 수: 2 )



넥스트 테슬라? 메타버스株 스치기만 해도 오른다 ( 댓글 수: 2 )

26


{0: {'기사명': "벤츠 E클래스, 7월 '수입차 1위' 탈환…테슬라는 물량 급감",
  '날짜': '2021.08.03. 오후 3:29',
  '추천수': '534',
  '좋아요': '10',
  '훈훈해요': '1',
  '슬퍼요': '2',
  '화나요': '16',
  '후속기사 원해요': '3',
  '댓글 수': 36,
  '댓글': ['한국내 벤츠 생산 공장을 만드는게 좋을듯 함. 물량도 되고 부품공급사들도 포진해있고..벤츠를 사랑하는 로열 고객 많고 품질도 더 좋아질수 있고 등등',
   '내돈내산 왜 욕하는지 왤케 열등감 쩔어 사는 애들이 많냐...',
   '결국 테슬라가 물량만 확보되면 1위라는 얘기.',
   '승차감, 정숙성, 가성비따위는 필요없다. 단지 벤츠이기에 벤츠를 타는거다.',
   '판매량 집계할땐 제네시스도 현대차, 기아차는 별도로 ㅎ',
   '테슬라는 수요를 못따라가는 겁니다. 판매가 저조한게 아니라...ㅎㅎㅎ',
   '쌍용차가 르노나 지엠보다 많이 팔렸네?',
   'E클래스 좋은가요?',
   '테슬라를 타고부터벤츠는 안녕...',
   '전기차만 아니면 된다',
   '테슬라!딱히 내장도 매력없고,외관은 밋밋하고 그렇다고 자동차 명가이미지도 없고 즉 브랜드력이 없고 그렇다고 자율주행이 완전 수준높은 레벨도 아니고 차라리 국산 제네시스 전기차나 벤츠의 전기자동차 타는게 훨씬 매력있는 선택!특히 명품가방처럼 중국보다 비싸게 한국소비자를 호갱으로 보는 나쁜 정책가진 테슬라는 국내에서 완전 추방시켜버려야',
   '전기차 많아지는데 굳이 테슬라 살까.. 나도 테슬라 패싱히느라 버티는 중.',
   '클린봇이 부적절한 표현을 감지한 댓글입니다.',
   '민노총 망하라고 벤즈 구입했다',
   '노조가 밥말아 먹는다 지들배만 부르면 만사땡',
   '국내만이자나 전세계인줄 어그로',
   '코로나고 나발이고. 당하는사람만 당하는거지',
   '삼각별 하나에 다 포기해도 좋다',
   '작성자에 의해 삭제된 댓글입니다.

In [10]:
# 판다스화
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, 'index')
result_df

,기사명,날짜,추천수,좋아요,훈훈해요,슬퍼요,화나요,후속기사 원해요,댓글 수,댓글
0,"벤츠 E클래스, 7월 '수입차 1위' 탈환…테슬라는 물량 급감",2021.08.03. 오후 3:29,534,10,1,2,16,3,36,[한국내 벤츠 생산 공장을 만드는게 좋을듯 함. 물량도 되고 부품공급사들도 포진해있...
1,테슬라 7월 판매 22대로 급감…'벤츠 E클래스' 2567대로 1위(종합),2021.08.03. 오후 8:09,207,3,2,0,8,0,21,[야 이제 이런 조작 기사도 지겹다 ㅋㅋㅋㅋ 걍 연단위로 보자. 연 외제차 수입 톱...
2,지난달 수입차 판매 1위는 '벤츠 E클래스'…테슬라는 부진 왜,2021.08.03. 오후 5:04,207,1,0,1,0,0,1,[벤츠는 점점 못생겨 가냐.]
3,테슬라 주춤한 사이 벤츠 날았다…7월에도 수입차 질주 여전,2021.08.03. 오후 1:21,"1,332",0,0,0,1,0,1,[결론은 테슬라는 한국에 들어오데로 다 팔려서 물량이 부족하다는거네. 없어서 못파는...
4,테슬라 문 안 열려 '중대결함'…머스크 서울서 고발당했다,2021.08.03. 오후 3:37,"2,234",12,1,0,6,0,16,"[응 테슬라 주가 영향 1도없죠~, 아무도 사지마라, 테슬라 고발할때 롤린춤추는거 ..."
5,[단독]'테슬라 중대결함' 시민단체가 머스크 고발...경찰 수사 나선다,2021.08.03. 오후 2:24,8,13,2,0,32,1,44,"[클린봇이 부적절한 표현을 감지한 댓글입니다., 소말리아의 한 시민단체가 한국 삼성..."
6,"경찰, '테슬라 중대결함' 머스크 고발 수사 착수",2021.08.03. 오후 3:20,769,0,0,0,3,0,4,"[흠... 과연,,, ㅋㅋ현다이는?ㅋㅋ, 소비자를 기만하는 기업의 악덕행위는 없어져..."
7,"""테슬라 중대 결함 속이고 판매""…일론 머스크 고발 사건 경찰이 수사",2021.08.03. 오후 4:53,"2,614",1,0,0,1,0,0,[]
8,'테슬라 중대결함'…일론 머스크 고발 件 서울경찰이 수사,2021.08.03. 오후 4:18,,0,0,0,0,0,2,[몇년전부터 문제있다고 나왔던 시스템같은데.. 이것도 소비자 단체에서 고발하는군요....
9,濠 테슬라 메가팩 배터리 화재 진화에만 나흘 걸려,2021.08.03. 오전 9:12,228,0,0,0,3,1,2,[ㅈ 되었네. 안그래도 사고 자주 나는 테슬라인데 이 사건 계기로 시장 점유율 더 ...


In [11]:
# 엑셀 저장
result_df.to_excel("naver_news_({}).xlsx".format(keyword))